In [2]:
source("utils/plot.R")

In [6]:
feat_factor <- function(v) {
    result <- str_replace_all(v, c(
        "pi"="Pi",
        "num_snps"="# SNPs",
        "num_haps"="# Haplotypes",
        "taj_D"="Tajima's D"
    ))
    result <- factor(result, levels=c("Pi", "# SNPs", "# Haplotypes", "H1", "H12", "H2/H1", "Tajima's D"))
    return(result)
}

In [49]:
signals <- read_tsv(snakemake@input[[1]], col_types=cols(selection_coefficient=col_number())) %>%
    mutate(
        feature=feat_factor(feature),
        position = position - 10,
        PositionInKbp = position*(window_size/2)/1000,
        WindowSize=paste(round(window_size/1000, 1), 'kb', sep=''),
        WindowSize=factor(WindowSize, levels=str_sort(unique(WindowSize), numeric=TRUE))
    )

In [50]:
# Pick K window sizes to use

K <- 5
acceptable_ws <- quantile(signals$window_size, probs=c(0, 0.25, 0.5, 0.75, 1))
signals <- signals %>%
    filter(window_size %in% acceptable_ws)

## Empirical signatures

In [1]:
empirical <- signals %>%
    filter(dataset != "simulated-hard-sweep") %>%
    separate(dataset, sep="_", into=c('dataset', NA, NA, NA)) %>%
    separate(dataset, sep='-', into=c(NA, 'Locus')) %>%
    mutate(Locus=str_replace_all(Locus, c("cyp"="Cyp6g1", "ace"="Ace", "chkov"="CHKov1")))

ERROR: Error in signals %>% filter(dataset != "simulated-hard-sweep") %>% separate(dataset, : could not find function "%>%"


In [52]:
empirical_fig <- ggplot(empirical) +
        geom_line(aes(x=PositionInKbp, y=value, colour=feature)) +
        facet_grid(rows=vars(Locus), cols=vars(WindowSize), switch='y', scales='free_x') +
        labs(
            x='Position (kb)',
            y='Normalized statistic'
        ) +
        sweeps_colour +
        sweeps_theme +
        theme(
            panel.grid.major.y = element_blank(),
            axis.text.y = element_blank(),
            axis.title.y = element_blank(),
            strip.text.y.left = element_text(angle = 0),
            panel.border = element_rect(colour='grey', fill=NA),
            legend.title = element_blank(),
            axis.text.x = element_text(angle=45, hjust=1),
            panel.spacing.x = unit(0.4, "lines"),
        )

In [53]:
empirical_fig

In [54]:
sweeps_save(snakemake@output$empirical, empirical_fig, width=6, asp=1.618)

## Simulated signatures

In [81]:
simulated <- signals %>%
    filter(dataset == "simulated-hard-sweep") %>%
    mutate(
        SelLabel = paste0('s=', selection_coefficient),
        SelLabel = fct_reorder(SelLabel, selection_coefficient)
    )

In [82]:
simulated_fig <- ggplot(simulated) +
        geom_line(aes(x=PositionInKbp, y=value, colour=feature)) +
        facet_grid(rows=vars(SelLabel), cols=vars(WindowSize), switch='y', scales='free_x') +
        labs(
            x='Position (kb)',
            y='Normalized statistic'
        ) +
        sweeps_colour +
        sweeps_theme +
        theme(
            panel.grid.major.y = element_blank(),
            axis.text.y = element_blank(),
            axis.title.y = element_blank(),
            strip.text.y.left = element_text(angle = 0),
            panel.border = element_rect(colour='grey', fill=NA),
            legend.title = element_blank(),
            axis.text.x = element_text(angle=45, hjust=1),
            panel.spacing.x = unit(0.4, "lines"),
        )

In [83]:
simulated_fig

In [86]:
sweeps_save(snakemake@output$simulated, simulated_fig, width=6, asp=1.3)